## 뉴욕시의 Taxi 데이터를 활용한 예측 모델
- 데이터셋 : 캐글의 NYT_taxi.csv
- 조건 : 뉴욕시 내
- '거리'에 따른 '요금' '예측' => 회귀
- 승차에 따른 하차지역 예측   => 분류

### <span style = 'color : red'>  [ 각 컬럼에서 가지는 특징들, 컬럼 간의 관계, 불필요한 것은 제거 ] </span>

위도와 경도 -> [범주형] 승하차 지역, 시간대/요일
           -> [수치형] 승객수
요금       -> [수치형] 거리마다의 요금 예측

### [1] 데이터 로딩

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
FILE='./NYC_taxi_data/train.csv'

In [3]:
# 데이터 로딩
# nrows=로딩 데이터 수
df=pd.read_csv(FILE, nrows=5000, parse_dates=['pickup_datetime'])

In [4]:
df.head()

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2009-06-15 17:26:21.0000001,4.5,2009-06-15 17:26:21+00:00,-73.844311,40.721319,-73.841610,40.712278,1
1,2010-01-05 16:52:16.0000002,16.9,2010-01-05 16:52:16+00:00,-74.016048,40.711303,-73.979268,40.782004,1
2,2011-08-18 00:35:00.00000049,5.7,2011-08-18 00:35:00+00:00,-73.982738,40.761270,-73.991242,40.750562,2
3,2012-04-21 04:30:42.0000001,7.7,2012-04-21 04:30:42+00:00,-73.987130,40.733143,-73.991567,40.758092,1
4,2010-03-09 07:51:00.000000135,5.3,2010-03-09 07:51:00+00:00,-73.968095,40.768008,-73.956655,40.783762,1


In [5]:
df.info()
# key값은 PrimaryKey로 쓰려고 들어간 것이고
# pickup_datetime => 컬럼과 같은 내용이므로 날리면 됨
# pickup_datetime => 문자열로 되어있으니, 읽어올 때 바로 바꿔줄 수도 있음
# =-> df=pd.read_csv(FILE, nrows=5000, parse_dates=['pickup_datetime'])에서 parse_dates부분

# 이런식으로 csv파일 열어보는 것은
# - 데이터 타입등을 확인하고 처리하기 위해서

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype              
---  ------             --------------  -----              
 0   key                5000 non-null   object             
 1   fare_amount        5000 non-null   float64            
 2   pickup_datetime    5000 non-null   datetime64[ns, UTC]
 3   pickup_longitude   5000 non-null   float64            
 4   pickup_latitude    5000 non-null   float64            
 5   dropoff_longitude  5000 non-null   float64            
 6   dropoff_latitude   5000 non-null   float64            
 7   passenger_count    5000 non-null   int64              
dtypes: datetime64[ns, UTC](1), float64(5), int64(1), object(1)
memory usage: 312.6+ KB


### '거리'에 따른 '요금', '예측' => 회귀

In [6]:
df.columns

Index(['key', 'fare_amount', 'pickup_datetime', 'pickup_longitude',
       'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude',
       'passenger_count'],
      dtype='object')

In [7]:
# 특성을 잘 뽑아내야 차원이 줄고 / 연산이 줄고 속도도 / 빨라지고

# 특성공학()
# - 결과에 중요한 영향을 미치는 피처 선택
# - 비슷한 피처 제거
# - 존재하지 않는 피처는 조합 하여 피처 생성

# 거리에 따른 요금을 알고 싶음
# => 특성(features)을 추가해야지 (기존 컬럼 활용해도 되지)

In [8]:
# 수치가 0이거나 결측치이거나 이상치이거나

### [2] 결측치 및 이상치 체크

In [9]:
df.isnull().sum()

key                  0
fare_amount          0
pickup_datetime      0
pickup_longitude     0
pickup_latitude      0
dropoff_longitude    0
dropoff_latitude     0
passenger_count      0
dtype: int64

In [10]:
# 데이터들의 분포
df.describe()

# i) 요금에 min에 -값이 있네!?
# ii) 승하차 지점은 있는데 승객은 없네!?
# iii) 위도, 경도 데이터에 0은 없네

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,11.296238,-72.426826,39.867237,-72.426130,39.875767,1.631000
std,9.710106,10.722381,6.351342,10.722293,6.326084,1.257122
min,-2.900000,-74.438233,-73.991840,-74.429332,-73.979010,0.000000
25%,6.000000,-73.992449,40.734045,-73.991349,40.734379,1.000000
50%,8.500000,-73.982069,40.752128,-73.980196,40.753512,1.000000
75%,12.500000,-73.967040,40.766890,-73.963589,40.767636,2.000000
max,180.000000,40.751582,41.366138,40.758867,41.032598,6.000000


#### 특성 새롭게 생성 => 거리 Feature
- 공식 : (x2-x1)^2 + (y2-y1)^2 계산 결과의 제곱근

In [16]:
# 값 뽑기
df.pickup_latitude
# df['pickup_latitude']

0       40.721319
1       40.711303
2       40.761270
3       40.733143
4       40.768008
          ...    
4995    40.745157
4996    40.766989
4997    40.734343
4998    40.776642
4999    40.743722
Name: pickup_latitude, Length: 5000, dtype: float64

In [12]:
df['distance'] = ((df.pickup_latitude - df.dropoff_latitude)**2 + (df.pickup_longitude - df.dropoff_longitude)**2)**0.5

# df['distance'] = distance
# sqlt((pickup_latitude - dropoff_latitude)^2 + (pickup_longitude - dropoff_longitude)^2)


In [13]:
df.columns

Index(['key', 'fare_amount', 'pickup_datetime', 'pickup_longitude',
       'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude',
       'passenger_count', 'distance'],
      dtype='object')

In [14]:
# 머신러닝 & 딥러닝 -> 유사도多
# - 숫자값 간의 거리를 보고 유사한 정도를 표현
# - 거리 계산 : 유클리드(多, 디폴트값)/맨하튼/코사인/해밍/자카드 거리법 등

# 데이터의 위도와 경도값을 알아도 사이의 거리값을 구해야 함
# 구글맵의 뉴욕시 눌러가면서
# 범위 벗어 나는 것 제거